In [1]:
!pip install --force /code/pgrtk-0.3.2-cp38-cp38-linux_x86_64.whl

Processing /code/pgrtk-0.3.1-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: pgrtk
    Found existing installation: pgrtk 0.3.1
    Uninstalling pgrtk-0.3.1:
      Successfully uninstalled pgrtk-0.3.1


In [2]:
import pgrtk
import os
import matplotlib.pyplot as plt
from IPython.display import SVG
%matplotlib inline

In [3]:
pgrtk.__version__

'pgrtk 0.3.1 (docs:e7ec376, release build, linux [x86_64] [rustc 1.62.0 (a8314ef7d 2022-06-27)])'

In [4]:
os.chdir("/results")

In [6]:
%%time
sdb = pgrtk.SeqIndexDB()
sdb.load_from_fastx("/results/output_dir/out_seqs_with_padding.fa")

CPU times: user 3min 47s, sys: 29.6 s, total: 4min 17s
Wall time: 25.2 s


In [7]:
seq_info = sdb.seq_info.copy()

In [8]:
len(seq_info)

37638

In [64]:
sample_to_seqeunces = {}
for sid in seq_info:
    ctg, src, _ = seq_info[sid]
    #print(ctg)
    gene, seq_name = ctg.split("_",1)
    #if gene not in ("SNTG2", "SLC27A5"):
    #    continue
    try:
        sample, haplotype, _ = seq_name.split("#", 2)
        haplotype = int(haplotype) - 1
        sample_to_seqeunces.setdefault(sample, [[],[]])
        seq = pgrtk.u8_to_string(sdb.get_seq(src, ctg))
        sample_to_seqeunces[sample][haplotype].append( (ctg, seq) )
        #print(sample, haplotype)
    except:
        pass
    

In [65]:
!mkdir -p /results/output2/

In [66]:
for sample in sample_to_seqeunces:
    for haplotype in (0, 1):
        with open("/results/output2/{}_{}_CMRG.fa".format(sample, haplotype), "w") as f:
            for ctg, seq in sample_to_seqeunces[sample][haplotype]:
                print(f">{ctg}", file=f)
                print(f"{seq}", file=f)

Install `dipcall` with `conda install -c bioconda dipcall`

In [40]:
!/code/agc-1.1_x64-linux/agc getset /data/grch38.agc  GCA_000001405.15_GRCh38_no_alt_analysis_set.fna > /results/grch38.fa

In [ ]:
%%bash
cd /results/output2/
minimap2 -d grch38.mmidx ../grch38.fa 
samtools faidx ../grch38.fa

In [67]:
%%writefile /results/output2/run_dipcall
#!/usr/bin/env perl

use strict;
use warnings;
use Getopt::Std;

my $version = "0.3";

my %opts = (t=>8);
getopts("t:d:x:muz:W:a", \%opts);

die("Usage: run-dipcall [options] <prefix> <ref.fa> <pat.fa> <mat.fa>
Options:
  -t INT     number of threads [$opts{t}]
  -d FILE    unimap/minimap2 index for ref.fa []
  -a         call on all contigs regardless of naming
  -x FILE    PAR on chrX; assuming male
  -z INT     Z-drop [mapper default]
  -m         use minimap2 for mapping (default)
  -u         use unimap for mapping
  -W FILE    repetitive k-mer list; use winnowmap for mapping
") if @ARGV < 4;

# test file existence
my $pre = shift(@ARGV);
my $ref = shift(@ARGV);
my @hap = @ARGV;
die "ERROR: failed to read the reference file '$ref'\n" unless -f $ref;
die "ERROR: please index the reference with 'samtools faidx'\n" unless -f "$ref.fai";
die "ERROR: failed to read the 1st haplotype '$hap[0]'\n" unless -f $hap[0];
die "ERROR: failed to read the 2nd haplotype '$hap[1]'\n" unless -f $hap[1];
my $is_male = defined($opts{x})? 1 : 0;
if (defined $opts{x}) {
	die "ERROR: failed to read PAR\n" unless -f $opts{x};
	$is_male = 1;
}

# find the root directory
my $exepath = $0 =~/^\S+\/[^\/\s]+/? $0 : &which($0);
#my $root = $0 =~/^(\S+)\/[^\/\s]+/? $1 : undef;
#$root = $exepath =~/^(\S+)\/[^\/\s]+/? $1 : undef if !defined($root);
my $root = "/opt/conda/bin/";
die "ERROR: failed to locate the root directory\n" if !defined($root);
# mapper settings
my $mapper = "minimap2";
my $mm2_opt = q/-xasm5 --cs -t$(N_THREADS) -z200000,10000/;
if (defined $opts{u}) {
	$mapper = "unimap";
} elsif (defined $opts{W}) {
	$mapper = "winnowmap";
	$mm2_opt .= " -r2k -W $opts{W}";
}
$mapper = "minimap2" if defined($opts{m});
$mm2_opt .= " -z$opts{z},200" if defined($opts{z});
my $mm2_idx = defined($opts{d})? $opts{d} : $ref;

# write Makefile
my @mak = ();

push(@mak, "ROOT=$root");
push(@mak, "N_THREADS=$opts{t}");
push(@mak, "MAPPER=$mapper");
push(@mak, "MM2_IDX=$mm2_idx");
push(@mak, "MM2_OPT=$mm2_opt");
push(@mak, "REF_FA=$ref");
push(@mak, "");

push(@mak, "all:$pre.dip.bed $pre.dip.vcf.gz", "");

push(@mak, "$pre.hap1.paf.gz:$hap[0]");
push(@mak, q{	$(ROOT)/$(MAPPER) -c --paf-no-hit $(MM2_OPT) $(MM2_IDX) $< 2> $@.log | gzip > $@});
push(@mak, "$pre.hap2.paf.gz:$hap[1]");
push(@mak, q{	$(ROOT)/$(MAPPER) -c --paf-no-hit $(MM2_OPT) $(MM2_IDX) $< 2> $@.log | gzip > $@});
push(@mak, "");

push(@mak, "$pre.hap1.sam.gz:$hap[0]");
push(@mak, q{	$(ROOT)/$(MAPPER) -a $(MM2_OPT) $(MM2_IDX) $< 2> $@.log | gzip > $@});
push(@mak, "$pre.hap2.sam.gz:$hap[1]");
push(@mak, q{	$(ROOT)/$(MAPPER) -a $(MM2_OPT) $(MM2_IDX) $< 2> $@.log | gzip > $@});
push(@mak, "");

push(@mak, "$pre.hap1.bam:$pre.hap1.sam.gz");
push(@mak, q{	$(ROOT)/k8 $(ROOT)/dipcall-aux.js samflt $< | $(ROOT)/samtools sort -m4G --threads 4 -o $@ -});
push(@mak, "$pre.hap2.bam:$pre.hap2.sam.gz");
push(@mak, q{	$(ROOT)/k8 $(ROOT)/dipcall-aux.js samflt $< | $(ROOT)/samtools sort -m4G --threads 4 -o $@ -});
push(@mak, "");

push(@mak, "$pre.pair.vcf.gz:$pre.hap1.bam $pre.hap2.bam");
push(@mak, q{	$(ROOT)/htsbox pileup -q5 -evcf $(REF_FA) $^ | $(ROOT)/htsbox bgzip > $@});
push(@mak, "$pre.dip.vcf.gz:$pre.pair.vcf.gz");

my $pair_opt = defined($opts{a})? "-a" : "";
if ($is_male) {
	push(@mak, q{	$(ROOT)/k8 $(ROOT)/dipcall-aux.js vcfpair} . qq{ $pair_opt -p $opts{x} } . q{$< | $(ROOT)/htsbox bgzip > $@});
} else {
	push(@mak, q{	$(ROOT)/k8 $(ROOT)/dipcall-aux.js vcfpair} . qq{ $pair_opt } . q{$< | $(ROOT)/htsbox bgzip > $@});
}
push(@mak, "");

push(@mak, "$pre.hap1.var.gz:$pre.hap1.paf.gz");
push(@mak, q{	gzip -dc $< | sort -k6,6 -k8,8n | $(ROOT)/k8 $(ROOT)/paftools.js call - 2> $@.vst | gzip > $@});
push(@mak, "$pre.hap2.var.gz:$pre.hap2.paf.gz");
push(@mak, q{	gzip -dc $< | sort -k6,6 -k8,8n | $(ROOT)/k8 $(ROOT)/paftools.js call - 2> $@.vst | gzip > $@});
push(@mak, "$pre.hap1.bed:$pre.hap1.var.gz");
push(@mak, q{	gzip -dc $< | grep ^R | cut -f2- > $@});
push(@mak, "$pre.hap2.bed:$pre.hap2.var.gz");
push(@mak, q{	gzip -dc $< | grep ^R | cut -f2- > $@});
push(@mak, "$pre.dip.bed:$pre.hap1.bed $pre.hap2.bed");
if ($is_male) {
	my $bedtk = '$(ROOT)/bedtk';
	my @cmd;
	push(@cmd, "$bedtk isec $pre.hap1.bed $pre.hap2.bed | egrep -v '^(chr)?[XY]' > $pre.tmp.bed"); # autosome
	push(@cmd, "$bedtk isec $pre.hap1.bed $pre.hap2.bed | $bedtk isec $opts{x} >> $pre.tmp.bed");  # chrX PAR
	push(@cmd, "$bedtk sub $pre.hap2.bed $pre.hap1.bed | egrep '^(chr)?X' | $bedtk sub - $opts{x} >> $pre.tmp.bed"); # chrX non-PAR
	push(@cmd, "$bedtk sub $pre.hap1.bed $pre.hap2.bed | egrep '^(chr)?Y' >> $pre.tmp.bed");       # chrY non-PAR
	push(@cmd, "$bedtk sort $pre.tmp.bed > " . '$@');
	push(@mak, "\t" . join("; ", @cmd));
} else {
	push(@mak, q{	$(ROOT)/bedtk isec -m $^ > $@});
}
push(@mak, "");

print(join("\n", @mak));

exit(0);

# auxiliary routines
sub which {
	my $file = shift;
	my $path = (@_)? shift : $ENV{PATH};
	return if (!defined($path));
	foreach my $x (split(":", $path)) {
		$x =~ s/\/$//;
		return "$x/$file" if (-x "$x/$file");
	}
	return;
}

Overwriting /results/output2/run_dipcall


In [68]:
!chmod u+x /results/output2/run_dipcall

In [69]:
os.chdir("/results/output2/")
for sample in sample_to_seqeunces:
    os.system(f"./run_dipcall -d grch38.mmidx DPC_{sample} ../grch38.fa {sample}_0_CMRG.fa {sample}_1_CMRG.fa > makefile_{sample}")

In [63]:
%%bash
cd /results/output2/
ls makefile_* | awk '{print "make -f "$1}' | parallel -j 1

Process is interrupted.


In [ ]:
%%bash
cd /results/
tar zcvf all_CMRG_dipcalls_GRCH38.tgz output2




